In [1]:
import pandas as pd
from flask_mysqldb import MySQL
from flask import Flask
from flask_cors import CORS
import random
import pymysql
from sqlalchemy import create_engine

# mission:
# 1. Load the products csv
# 2. get the needed columns
# 3. insert into database with generating all needed fields for the ML
# 4. make an aggregated csv

## Setting product table

In [2]:
data = pd.read_csv("DatafinitiElectronicsProductsPricingData.csv")

In [3]:
data = data[['dateAdded', 'prices.amountMin', 'imageURLs', 'brand', 'manufacturerNumber', 'name', 'dateUpdated','categories']]
data.columns  = ['date_added', 'price', 'image', 'brand', 'model', 'name', 'date_updated', 'category']

In [4]:
data['date_added'] = data['date_added'].apply(lambda s: s[:10])
data['date_updated'] = data['date_updated'].apply(lambda s: s[:10])
data.head()

,date_added,price,image,brand,model,name,date_updated,category
0,2015-04-13,104.99,https://images-na.ssl-images-amazon.com/images...,Sanus,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,2018-05-12,"Audio & Video Accessories,TV Mounts,TV Accesso..."
1,2015-05-18,64.99,https://images-na.ssl-images-amazon.com/images...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,2018-06-13,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
2,2015-05-18,69.00,https://images-na.ssl-images-amazon.com/images...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,2018-06-13,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
3,2015-05-18,69.99,https://images-na.ssl-images-amazon.com/images...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,2018-06-13,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
4,2015-05-18,66.99,https://images-na.ssl-images-amazon.com/images...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,2018-06-13,"Stereos,Portable Bluetooth Speakers,TV, Video ..."


In [5]:
quantity = [random.randint(0,1000) for x in range(data.shape[0])]
discount_amount = [random.randint(0,100) for x in range(data.shape[0])]
for r in random.sample(range(0,14592), 6000):
    discount_amount[r] = 0

In [6]:
data.insert(loc = 3, column = 'quantity', value = quantity)
data.insert(loc=7, column='discount_amount', value=discount_amount)
data = data.drop_duplicates(subset=['name'], keep='first')
data.head()

,date_added,price,image,quantity,brand,model,name,discount_amount,date_updated,category
0,2015-04-13,104.99,https://images-na.ssl-images-amazon.com/images...,290,Sanus,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,10,2018-05-12,"Audio & Video Accessories,TV Mounts,TV Accesso..."
1,2015-05-18,64.99,https://images-na.ssl-images-amazon.com/images...,537,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,6,2018-06-13,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
13,2015-03-18,23.99,https://pisces.bbystatic.com/image2/BestBuy_US...,571,DENAQ,DQ-PA3032U-5525,DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 ...,0,2018-06-13,"Computers,Office,Computer Accessories,Computer..."
18,2015-10-17,290.99,https://pisces.bbystatic.com/image2/BestBuy_US...,65,DreamWave,TREMOR,DreamWave - Tremor Portable Bluetooth Speaker ...,23,2018-05-29,"Portable Bluetooth Speakers,Bluetooth & Wirele..."
23,2015-07-03,244.01,http://pisces.bbystatic.com/image2/BestBuy_US/...,379,Yamaha,NSSP1800BL,NS-SP1800BL 5.1-Channel Home Theater System (B...,91,2018-07-26,"Surround Speakers,Electronics Features,Surroun..."


In [7]:
data['image'] = data['image'].apply(lambda i:i[:i.find(',')])
data.head()

,date_added,price,image,quantity,brand,model,name,discount_amount,date_updated,category
0,2015-04-13,104.99,https://images-na.ssl-images-amazon.com/images...,290,Sanus,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,10,2018-05-12,"Audio & Video Accessories,TV Mounts,TV Accesso..."
1,2015-05-18,64.99,https://images-na.ssl-images-amazon.com/images...,537,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,6,2018-06-13,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
13,2015-03-18,23.99,https://pisces.bbystatic.com/image2/BestBuy_US...,571,DENAQ,DQ-PA3032U-5525,DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 ...,0,2018-06-13,"Computers,Office,Computer Accessories,Computer..."
18,2015-10-17,290.99,https://pisces.bbystatic.com/image2/BestBuy_US...,65,DreamWave,TREMOR,DreamWave - Tremor Portable Bluetooth Speaker ...,23,2018-05-29,"Portable Bluetooth Speakers,Bluetooth & Wirele..."
23,2015-07-03,244.01,http://pisces.bbystatic.com/image2/BestBuy_US/...,379,Yamaha,NSSP1800BL,NS-SP1800BL 5.1-Channel Home Theater System (B...,91,2018-07-26,"Surround Speakers,Electronics Features,Surroun..."


In [8]:
data.insert(loc=0, column='product_id', value=[i for i in range(1, data.shape[0]+1)])
data.head()
category_ = data.copy()
data.drop('category',axis=1,inplace=True)

In [34]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'DefinetlyNotAmazon' + '@' + 'localhost' + ':3306/' \
       + 'onlShop' + '?charset=utf8mb4'
engine = create_engine(db_data)
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='DefinetlyNotAmazon',
                         db='onlShop')    

# create cursor
cursor=connection.cursor()

In [35]:

# Execute the to_sql for writting DF into SQL
data.to_sql("product", 
          engine, 
          index=False, 
          if_exists='append',
          chunksize=25000,
          method=None)
# data.to_sql('product', engine, if_exists='append', index=False)

1229

In [36]:
connection.commit()
cursor.close()

In [37]:
# query1 = "SELECT * FROM Product NATURAL JOIN Views NATURAL JOIN customer"
# query2 = "SELECT COUNT(*) FROM order_ NATURAL JOIN containment NATURAL JOIN product"
# query3 = "SELECT country FROM customer"
# query4 = "SELECT * FROM VIEWS"

# cursor.execute(query1)
# result = cursor.fetchall()
# result

## Setting category_ table

In [9]:
category_ = category_[['product_id', 'category']]
category_.head()

,product_id,category
0,1,"Audio & Video Accessories,TV Mounts,TV Accesso..."
1,2,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
13,3,"Computers,Office,Computer Accessories,Computer..."
18,4,"Portable Bluetooth Speakers,Bluetooth & Wirele..."
23,5,"Surround Speakers,Electronics Features,Surroun..."


In [10]:
# category_.category = category_.category.apply(lambda x: pd.Series(str(x).split(",")))
test = pd.DataFrame(columns=category_.columns)
idx, id = 0, 1
for i in category_.category.str.split(',|&',expand=False):
    # test['product_id'] = idx
    for j in i:
        # test['category'] = j
        test.loc[idx] = [id, j.lower().strip()]
        # test.concat({'product_id': idx}, {'category': j})
        idx += 1
    id += 1

test.drop_duplicates(keep='first', inplace=True, ignore_index=True)
test.tail()

,product_id,category
20300,1229,theater
20301,1229,all home speakers
20302,1229,home theater systems
20303,1229,computer speakers
20304,1229,mp3 accessories


In [40]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'DefinetlyNotAmazon' + '@' + 'localhost' + ':3306/' \
       + 'onlShop' + '?charset=utf8mb4'
engine = create_engine(db_data)
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='DefinetlyNotAmazon',
                         db='onlShop')    

# create cursor
cursor=connection.cursor()

test.to_sql("category_", 
          engine, 
          index=False, 
          if_exists='append',
          chunksize=25000,
          method=None)

connection.commit()
cursor.close()

## Setting customer table

In [129]:
d = pd.read_csv("20221231151259_7530.csv")
d.columns = ['Fname','Lname','email','balance', 'password']
d.head()

,Fname,Lname,email,balance,password
0,Miley,Gibson,m.gibson@randatmail.com,157024,0aj71h2
1,Arnold,Watson,a.watson@randatmail.com,122281,ln8kv88
2,Robert,Moore,r.moore@randatmail.com,83842,v6it2bu
3,Miller,Foster,m.foster@randatmail.com,83166,dpapb8a
4,Kevin,Murphy,k.murphy@randatmail.com,177955,lq6527d


In [130]:
from datetime import date, timedelta
import random
  
# initializing dates ranges 
test_date1, test_date2 = date(1980, 1, 1), date(2002, 1, 1)
  
# initializing K
K = d.shape[0]
  
# getting days between dates
dates_bet = test_date2 - test_date1
total_days = dates_bet.days
  
res = []
for idx in range(K):
    random.seed(a=None)
      
    # getting random days
    randay = random.randrange(total_days)
      
    # getting random dates 
    res.append(test_date1 + timedelta(days=randay))
  
# printing 
print(res[0])

1996-01-04


In [131]:
d['Bdate'] = res
d.head()

,Fname,Lname,email,balance,password,Bdate
0,Miley,Gibson,m.gibson@randatmail.com,157024,0aj71h2,1996-01-04
1,Arnold,Watson,a.watson@randatmail.com,122281,ln8kv88,1985-12-29
2,Robert,Moore,r.moore@randatmail.com,83842,v6it2bu,1989-08-08
3,Miller,Foster,m.foster@randatmail.com,83166,dpapb8a,1988-05-18
4,Kevin,Murphy,k.murphy@randatmail.com,177955,lq6527d,1985-03-12


In [23]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'DefinetlyNotAmazon' + '@' + 'localhost' + ':3306/' \
       + 'onlShop' + '?charset=utf8mb4'
engine = create_engine(db_data)
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='DefinetlyNotAmazon',
                         db='onlShop')    

# create cursor
cursor=connection.cursor()

d.to_sql("customer", 
          engine, 
          index=False, 
          if_exists='append',
          chunksize=25000,
          method=None)

connection.commit()
cursor.close()

## Setting views table

In [13]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'DefinetlyNotAmazon' + '@' + 'localhost' + ':3306/' \
       + 'onlShop' + '?charset=utf8mb4'
engine = create_engine(db_data)
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='DefinetlyNotAmazon',
                         db='onlShop')  
cursor=connection.cursor()
p_query = "SELECT product_id FROM product"
cursor.execute(p_query)
p_result = cursor.fetchall()

c1 = pd.DataFrame(p_result, columns=['product_id'])
c2 = pd.DataFrame([random.randint(201,400) for i in range(c1.shape[0])], columns=['user_id'])
c3 = pd.concat([c1, c2], axis=1)
c3['times_viewed'] = [random.randint(1,30) for i in range(c3.shape[0])]
c3

,product_id,user_id,times_viewed
0,1,245,25
1,2,201,12
2,3,399,19
3,4,311,22
4,5,303,29
...,...,...,...
1224,1225,220,29
1225,1226,208,29
1226,1227,297,21
1227,1228,381,21


In [ ]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'DefinetlyNotAmazon' + '@' + 'localhost' + ':3306/' \
       + 'onlShop' + '?charset=utf8mb4'
engine = create_engine(db_data)
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='DefinetlyNotAmazon',
                         db='onlShop')    

# create cursor
cursor=connection.cursor()

c3.to_sql("views", 
          engine, 
          index=False, 
          if_exists='append',
          chunksize=25000,
          method=None)

connection.commit()
cursor.close()

In [119]:
data.head()

NameError: name 'data' is not defined

In [135]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'DefinetlyNotAmazon' + '@' + 'localhost' + ':3306/' \
       + 'onlShop' + '?charset=utf8mb4'
engine = create_engine(db_data)
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='DefinetlyNotAmazon',
                         db='onlShop')    

# create cursor
cursor=connection.cursor()
query = "SELECT * FROM customer c JOIN views v ON c.user_id=v.user_id JOIN product p ON p.product_id=v.product_id"
cursor.execute(query)
result = cursor.fetchall()
cursor.close()


In [136]:
ML = pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,201,m.gibson@randatmail.com,0aj71h2,157024.0,1999-09-07,Miley,Gibson,None,None,None,...,342,2015-09-10,189.99,https://i5.walmartimages.com/asr/22917a1e-d877...,Kanto,SUB6GB,866,Kanto - sub6 6 80W Powered Subwoofer - Gloss B...,13,2018-06-13
1,201,m.gibson@randatmail.com,0aj71h2,157024.0,1999-09-07,Miley,Gibson,None,None,None,...,573,2017-03-13,1089.95,http://i.ebayimg.com/thumbs/images/g/iisAAOSwj...,Samsung,qn65q6fnafxza,423,SAMSUNG 65 Class 4K (2160P) Ultra HD Smart QLE...,0,2018-06-25
2,201,m.gibson@randatmail.com,0aj71h2,157024.0,1999-09-07,Miley,Gibson,None,None,None,...,718,2015-08-18,64.75,https://i5.walmartimages.com/asr/2c0e6884-2187...,SanDisk,YD8345,433,SanDisk Extreme Pro 32 GB SDHC,0,2018-03-12
3,201,m.gibson@randatmail.com,0aj71h2,157024.0,1999-09-07,Miley,Gibson,None,None,None,...,731,2015-11-16,99.99,https://static.bhphoto.com/images/itemImgPlace...,Westone,78514,916,UM Pro10 Single-Driver Universal In-Ear Monito...,77,2018-01-28
4,201,m.gibson@randatmail.com,0aj71h2,157024.0,1999-09-07,Miley,Gibson,None,None,None,...,969,2017-07-18,1399.99,https://images-na.ssl-images-amazon.com/images...,Sony,ILCE7M2/B,687,Alpha a7 II Mirrorless Digital Camera Body wit...,0,2018-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,400,c.smith@randatmail.com,kdqv07j,137434.0,1988-02-05,Cherry,Smith,None,None,None,...,646,2015-09-11,149.99,https://www.barcodable.com/images/barcode/0602...,Westone,78551,147,UM Pro10 Single-Driver Universal In-Ear Monito...,4,2018-06-05
1225,400,c.smith@randatmail.com,kdqv07j,137434.0,1988-02-05,Cherry,Smith,None,None,None,...,665,2015-11-06,69.99,https://i5.walmartimages.com/asr/1be435f7-5f3a...,Sony,SRSX11/BLK,222,Sony Ultra-Portable Bluetooth Speaker,0,2018-01-30
1226,400,c.smith@randatmail.com,kdqv07j,137434.0,1988-02-05,Cherry,Smith,None,None,None,...,810,2017-03-06,199.95,https://pisces.bbystatic.com/image2/BestBuy_US...,Sennheiser,HD 599,824,Sennheiser - HD 599 Over-the-Ear Headphones HD...,0,2018-06-13
1227,400,c.smith@randatmail.com,kdqv07j,137434.0,1988-02-05,Cherry,Smith,None,None,None,...,1069,2015-09-11,53.26,https://i5.walmartimages.com/asr/97fcb240-185f...,HMDX,hx-p340gy,396,HMDX Dynamite Portable Splashproof Wireless Sp...,98,2018-01-30
